In [ ]:
with open("labeled.txt","r") as f:
  text=f.read().lower().split("\n")


# Model 1 0~1 Negative conf   Model 2 0~1 Positive conf
X1 = []
Y1 = []
X2 = []
Y2 = []
for i in text:
  if i == "":
    continue
  o = i.split("------")
  print(o)
  if o[1] == "":
    X1.append(o[0].split(" "))
    Y1.append(0)
    X2.append(o[0].split(" "))
    Y2.append(0)

In [260]:
with open("Positive.txt","r") as f:
  text=f.read().lower().replace("“","").replace("”","").replace('"',"").split("\n")
for i in text:
  if i == "":
    continue

  X1.append(i.split(" "))
  Y1.append(0)
  X2.append(i.split(" "))
  Y2.append(1)

In [261]:
with open("Negative.txt","r") as f:
  text=f.read().lower().replace("“","").replace("”","").replace('"',"").split("\n")
for i in text:
  if i == "":
    continue

  X1.append(i.split(" "))
  Y1.append(1)
  X2.append(i.split(" "))
  Y2.append(0)

In [ ]:
np.sum(Y1)

307

In [262]:
word_to_token = {}
token_count = 0
tokened = []

for i in X1:
  line = []
  for word in i:
    try:
      line.append(word_to_token[word])
    except:
      token_count+=1
      word_to_token[word]=token_count
      line.append(word_to_token[word])
  tokened.append(line)
max_length = 0
for i in tokened:
  if len(i)>max_length:
    max_length=len(i)

padded = []
for i in tokened:
  i.extend([0]*(max_length+1-len(i)))
  padded.append(i)

In [263]:
import pickle
with open("word_to_token.pkl","wb") as f:
  pickle.dump(word_to_token, f)

In [ ]:
import numpy as np

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

import matplotlib.pyplot as plt


In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(max_length+1)),
    tf.keras.layers.Embedding(
        input_dim=token_count+2,
        output_dim=64),
    # tf.keras.layers.Conv1D(16, 5, activation='relu'),
    # tf.keras.layers.Conv1D(32, 5, activation='relu'),
    # tf.keras.layers.Conv1D(64, 5, activation='relu'),
    # tf.keras.layers.Conv1D(128, 5, activation='relu'),
    tf.keras.layers.GRU(100, activation='relu', return_sequences=1),
    tf.keras.layers.Dense(50, activation='relu'),
    tf.keras.layers.Dense(25, activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(2048, activation='relu'),
    tf.keras.layers.Dense(1024, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.8),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(18, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.summary()

In [ ]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=["accuracy"])

In [ ]:
# model.fit(X, padded)
index = np.arange(0,len(X1))
np.random.shuffle(index)
padded = np.array(padded)[index]
Y1 = np.array(Y1)[index]
model.fit(padded, Y1, epochs=30, shuffle=True, validation_split=0.1)
# https://stackoverflow.com/questions/51581521/accuracy-stuck-at-50-keras
# THIS HELPED A LOT!!! A QUESTION THAT WAS ANNOYING ME FOR MORE THAN A YEAR!

Epoch 1/30
22/22 [==============================] - 10s 379ms/step - loss: 0.6900 - accuracy: 0.5650 - val_loss: 0.6836 - val_accuracy: 0.6104
Epoch 2/30
22/22 [==============================] - 8s 359ms/step - loss: 0.6814 - accuracy: 0.5925 - val_loss: 0.6660 - val_accuracy: 0.6104
Epoch 3/30
22/22 [==============================] - 8s 376ms/step - loss: 0.6818 - accuracy: 0.5896 - val_loss: 0.6675 - val_accuracy: 0.6104
Epoch 4/30
22/22 [==============================] - 8s 348ms/step - loss: 0.6660 - accuracy: 0.5939 - val_loss: 0.6447 - val_accuracy: 0.6104
Epoch 5/30
22/22 [==============================] - 8s 362ms/step - loss: 0.6522 - accuracy: 0.5968 - val_loss: 0.6183 - val_accuracy: 0.6104
Epoch 6/30
22/22 [==============================] - 8s 354ms/step - loss: 0.6337 - accuracy: 0.5997 - val_loss: 0.5790 - val_accuracy: 0.6234
Epoch 7/30
22/22 [==============================] - 8s 356ms/step - loss: 0.5940 - accuracy: 0.6460 - val_loss: 0.5209 - val_accuracy: 0.8052
Epoch

KeyboardInterrupt: ignored

In [ ]:
test = "this service will still tracks you even after your account is deleted"
tokened_test = [word_to_token[word] for word in test.split(" ")]
tokened_test.extend([0]*(max_length+1-len(tokened_test)))
model.predict([tokened_test])

1/1 [==============================] - 0s 56ms/step


array([[0.9596156]], dtype=float32)

In [ ]:
tokened_test

In [ ]:
word_to_token = {}
token_count = 0
tokened = []

for i in X2:
  line = []
  for word in i:
    try:
      line.append(word_to_token[word])
    except:
      token_count+=1
      word_to_token[word]=token_count
      line.append(word_to_token[word])
  tokened.append(line)
max_length = 0
for i in tokened:
  if len(i)>max_length:
    max_length=len(i)

padded2 = []
for i in tokened:
  i.extend([0]*(max_length+1-len(i)))
  padded2.append(i)

model2 = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(max_length+1)),
    tf.keras.layers.Embedding(
        input_dim=token_count+2,
        output_dim=64),
    # tf.keras.layers.Conv1D(16, 5, activation='relu'),
    # tf.keras.layers.Conv1D(32, 5, activation='relu'),
    # tf.keras.layers.Conv1D(64, 5, activation='relu'),
    # tf.keras.layers.Conv1D(128, 5, activation='relu'),
    tf.keras.layers.GRU(100, activation='relu', return_sequences=1),
    tf.keras.layers.Dense(50, activation='relu'),
    tf.keras.layers.Dense(25, activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(2048, activation='relu'),
    tf.keras.layers.Dense(1024, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.8),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(18, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model2.compile(loss=tf.keras.losses.BinaryCrossentropy(),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=["accuracy"])
index = np.arange(0,len(X2))
np.random.shuffle(index)
padded = np.array(padded2)[index]
Y2 = np.array(Y2)[index]

Epoch 1/30


KeyboardInterrupt: ignored

In [ ]:
history = model2.fit(padded, Y2, epochs=30, shuffle=True, validation_split=0.1)


Epoch 1/30
22/22 [==============================] - 8s 386ms/step - loss: 0.6921 - accuracy: 0.5506 - val_loss: 0.6881 - val_accuracy: 0.6623
Epoch 2/30
22/22 [==============================] - 8s 352ms/step - loss: 0.6821 - accuracy: 0.6084 - val_loss: 0.6566 - val_accuracy: 0.6623
Epoch 3/30
22/22 [==============================] - 8s 354ms/step - loss: 0.6683 - accuracy: 0.6127 - val_loss: 0.6420 - val_accuracy: 0.6623
Epoch 4/30
22/22 [==============================] - 8s 353ms/step - loss: 0.6611 - accuracy: 0.6228 - val_loss: 0.6318 - val_accuracy: 0.6623
Epoch 5/30
22/22 [==============================] - 8s 357ms/step - loss: 0.6526 - accuracy: 0.6214 - val_loss: 0.6181 - val_accuracy: 0.6623
Epoch 6/30
22/22 [==============================] - 8s 363ms/step - loss: 0.6448 - accuracy: 0.6228 - val_loss: 0.6069 - val_accuracy: 0.6623
Epoch 7/30
22/22 [==============================] - 8s 348ms/step - loss: 0.6323 - accuracy: 0.6142 - val_loss: 0.5801 - val_accuracy: 0.6623
Epoch 

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
tf = CountVectorizer()
X = tf.fit_transform(X1)

In [ ]:
X1

In [ ]:
model.save('model1')
model2.save('model2')


In [ ]:
!zip -r model1.zip model1
!zip -r model2.zip model2

  adding: model1/ (stored 0%)
  adding: model1/assets/ (stored 0%)
  adding: model1/variables/ (stored 0%)
  adding: model1/variables/variables.index (deflated 69%)
  adding: model1/variables/variables.data-00000-of-00001 (deflated 14%)
  adding: model1/keras_metadata.pb (deflated 92%)
  adding: model1/saved_model.pb (deflated 89%)
  adding: model2/ (stored 0%)
  adding: model2/assets/ (stored 0%)
  adding: model2/variables/ (stored 0%)
  adding: model2/variables/variables.index (deflated 69%)
  adding: model2/variables/variables.data-00000-of-00001 (deflated 11%)
  adding: model2/keras_metadata.pb (deflated 92%)
  adding: model2/saved_model.pb (deflated 89%)


In [ ]:
!ls -l -h

total 480M
drwx------ 6 root root 4.0K Nov 27 06:09 drive
-rw-r--r-- 1 root root  37K Nov 27 04:26 labeled.txt
drwxr-xr-x 4 root root 4.0K Nov 27 06:09 model1
-rw-r--r-- 1 root root 129M Nov 27 06:07 model1.h5
-rw-r--r-- 1 root root 110M Nov 27 06:10 model1.zip
drwxr-xr-x 4 root root 4.0K Nov 27 06:09 model2
-rw-r--r-- 1 root root 129M Nov 27 06:07 model2.h5
-rw-r--r-- 1 root root 114M Nov 27 06:10 model2.zip
-rw-r--r-- 1 root root  64K Nov 27 04:28 Negative.txt
-rw-r--r-- 1 root root  39K Nov 27 04:29 Positive.txt
drwxr-xr-x 1 root root 4.0K Nov 22 00:14 sample_data


In [ ]:
!cp model1.zip drive/MyDrive/
!cp model2.zip drive/MyDrive/
